In [6]:
%cd "/home/kera/workspace/Transformer-GB"

/data/kera/workspace/Transformer-GB


In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import torch
import yaml 
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import argparse
import re 
from transformers import AutoModelForSeq2SeqLM, AutoConfig

from transformers import PreTrainedTokenizerFast
from src.loader.data import load_data
from src.loader.checkpoint import load_trained_bag
from src.evalution.evaluators import eval_prediction

load('src/data/symbolic_utils.sage')

In [9]:
import pickle
import numpy, time 
n=2
for field in ['QQ']: #['QQ', 'F7', 'F31']:
    for n in [2, 3, 4, 5]:
        id = f'results/timing/gb_dataset_n={n}_field={field}'
        print(f'{field}, n={n}')

        load_path = f'results/shape_gb_lex/gb_dataset_n={n}_field={field}'
        bag = load_trained_bag(load_path, from_checkpoint=False)
        model = bag['model'] 
        tokenizer = bag['tokenizer']

        data_path = f'{id}/data'
        test_loader = load_data(data_path, encoding='prefix', batch_sizes=[int(1)], return_dataloader=False, extensions=['test.lex'], do_shuffle=[False], tokenizer=None)
        test_loader_raw = load_data(data_path, encoding='raw', batch_sizes=[int(1)], return_dataloader=False, extensions=['test.lex'], do_shuffle=[False], tokenizer=None)

        with open(os.path.join(id, 'success_table.pickle'), 'rb') as f:
            sucecss_table = pickle.load(f)
            for key in sucecss_table:
                failed_id = np.where(~sucecss_table[key])[0]
                if len(failed_id) > 0:
                    print(f'{key:20}: {failed_id}')
                else:
                    print(f'{key:20}: -')

                for fid in failed_id:
                    ## test by Transformer
                    x_text = test_loader[fid]['input']
                    y_text = test_loader[fid]['target']
                    x = tokenizer(x_text, return_tensors='pt')['input_ids'].cuda()
                    y = tokenizer(y_text, return_tensors='pt')['input_ids'].cuda()
                    start = time.time()
                    output_ids = model.generate(x, max_length=y.shape[-1], num_beams=1, do_sample=False)
                    end = time.time()

                    y_pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
                    print(f'[{fid:03}] Transformer -- {y_text == y_pred}, {end-start} sec')

                    ## test by Math
                    bfield = QQ if field == 'QQ' else GF(int(field[1:]))
                    ring = PolynomialRing(bfield, n, 'x', order='lex')

                    x_text = test_loader_raw[fid]['input']
                    F = [ring(xt) for xt in x_text.split('[SEP]')]
                    start = time.time()
                    ideal(F).groebner_basis(algorithm=key)
                    end = time.time()
                    print(f'[{fid:03}] {key} -- {end-start} sec')


                    # [prefix_to_poly(xt) for xt in x_text.split('[SEP]')]

            print('')

QQ, n=2


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


loading ... results/timing/gb_dataset_n=2_field=QQ/data.test.lex
loading ... results/timing/gb_dataset_n=2_field=QQ/data.test.lex
libsingular:std     : -
libsingular:slimgb  : -
libsingular:stdfglm : -

QQ, n=3


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


loading ... results/timing/gb_dataset_n=3_field=QQ/data.test.lex
loading ... results/timing/gb_dataset_n=3_field=QQ/data.test.lex
libsingular:std     : [267 707 788 833]


/home/kera/mambaforge/envs/sage/lib/python3.10/site-packages/optimum/bettertransformer/models/encoder_models.py:487: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, ~attention_mask)


[267] Transformer -- True, 0.6990737915039062 sec


KeyboardInterrupt: 

In [5]:
from pprint import pprint
import pickle
import numpy, time 
n=4
field = 'QQ'
id = f'results/timing/gb_dataset_n={n}_field={field}'
print(f'{field}, n={n}')

# load_path = f'results/shape_gb_lex/gb_dataset_n={n}_field={field}'
# bag = load_trained_bag(load_path, from_checkpoint=False)
# model = bag['model'] 
# tokenizer = bag['tokenizer']

data_path = f'{id}/data'
test_loader = load_data(data_path, encoding='prefix', batch_sizes=[int(1)], return_dataloader=False, extensions=['test.lex'], do_shuffle=[False], tokenizer=None)
test_loader_raw = load_data(data_path, encoding='raw', batch_sizes=[int(1)], return_dataloader=False, extensions=['test.lex'], do_shuffle=[False], tokenizer=None)

sample_dict = {}
tmp_F, tmp_G = [], []
with open(os.path.join(id, 'success_table.pickle'), 'rb') as f:
    sucecss_table = pickle.load(f)
    for algorithm in sucecss_table:
        failed_id = np.where(~sucecss_table[algorithm])[0]
        if len(failed_id) > 0:
            print(f'{algorithm:20}: {failed_id}')
        else:
            print(f'{algorithm:20}: -')

        for fid in failed_id:
            ## test by Math
            bfield = QQ if field == 'QQ' else GF(int(field[1:]))
            ring = PolynomialRing(bfield, n, 'x', order='lex')

            x_text = test_loader_raw[fid]['input']
            y_text = test_loader_raw[fid]['target']
            F = [ring(xt) for xt in x_text.split('[SEP]')]
            G = [ring(yt) for yt in y_text.split('[SEP]')]
            sample_dict[fid] = (F, G)

            start = time.time()
            algorithm = 'libsingular:stdfglm'
            ideal(F).groebner_basis(algorithm=algorithm)
            end = time.time()
            print(f'[{fid:03}] {algorithm} -- {end-start} sec')

            new_ring = PolynomialRing(QQ, 4, 'x', order='degrevlex')
            G_drl = ideal(G).change_ring(new_ring).groebner_basis(algorithm='libsingular:stdfglm')

            print(f'- ID: {[fid]} --------')
            for g in G_drl:
                print(g)
            
            print(' ')
            # [prefix_to_poly(xt) for xt in x_text.split('[SEP]')]

    print('')

QQ, n=4
loading ... results/timing/gb_dataset_n=4_field=QQ/data.test.lex
loading ... results/timing/gb_dataset_n=4_field=QQ/data.test.lex
libsingular:std     : [ 19 313 578 648 705 797]
[019] libsingular:stdfglm -- 0.15416932106018066 sec
- ID: [19] --------
x0^2 - 61/36*x0 - 197/45*x1 - 56/45*x2 - 1741/360*x3 + 2771/900
x0*x1 + 151/72*x0 + 7003/720*x1 + 871/180*x2 + 55147/2880*x3 - 10193/1800
x1^2 - 865/144*x0 - 21133/1440*x1 - 2401/360*x2 - 163777/5760*x3 + 1838/225
x0*x2 - 239/180*x0 - 689/180*x1 - 77/180*x2 - 3071/720*x3 + 383/900
x1*x2 + 127/72*x0 + 1621/180*x1 + 26/9*x2 + 2453/144*x3 - 313/90
x2^2 - 1619/288*x0 - 45527/2880*x1 - 2951/720*x2 - 406883/11520*x3 - 145/36
x0*x3 - 1/6*x0 - 29/12*x1 - 5/3*x2 - 269/48*x3 + 11/6
x1*x3 + 19/12*x0 + 319/120*x1 + 43/30*x2 + 2251/480*x3 - 161/75
x2*x3 + 4/3*x0 + 77/60*x1 - 1/15*x2 + 881/240*x3 + 154/75
x3^2 - x0 - 13/10*x1 - 2/5*x2 - 97/40*x3 + 8/25
 
[313] libsingular:stdfglm -- 0.030172109603881836 sec
- ID: [313] --------
x0^2 + 22790989/3

In [9]:
new_ring = PolynomialRing(QQ, 4, 'x', order='degrevlex')

G_drl = ideal(G).change_ring(new_ring).groebner_basis(algorithm='libsingular:stdfglm')

In [7]:
for g in tmp_F:
    print(g)

1/5*x0^2*x2*x3^5 - 1/10*x0^2*x2*x3^3 + 1/15*x0^2*x2*x3 - 5/4*x1*x2^3 + 5/2*x2^3*x3^4 - 5/8*x2^3*x3^2 - 5/6*x2^3
3/20*x0^2*x1^3*x2*x3^5 - 3/40*x0^2*x1^3*x2*x3^3 + 1/20*x0^2*x1^3*x2*x3 - 15/16*x1^4*x2^3 + 15/8*x1^3*x2^3*x3^4 - 15/32*x1^3*x2^3*x3^2 - 5/8*x1^3*x2^3 + x1 - 2*x3^4 + 1/2*x3^2 + 2/3
-4/5*x0^2*x1 + 8/5*x0^2*x3^4 - 2/5*x0^2*x3^2 - 8/15*x0^2 - x0*x1*x3 - 3/4*x1*x3^5 - 1/2*x1*x3^4 + 5/4*x1*x3 + x2 + 2/3*x3^4 - 4/5*x3^3 - 2/3
x0 + 3/2*x1*x2 - 3*x2*x3^4 + 3/4*x2*x3^2 + x2 + 3/4*x3^4 + 1/2*x3^3 - 5/4
-3*x1^2*x2*x3 - 2*x1^2*x3^5 + 12/5*x1^2*x3^4 + 2*x1^2*x3 + x3^5 - 1/2*x3^3 + 1/3*x3
0


In [17]:
for g in tmp_G:
    print(g)

x0 + 3/4*x3^4 + 1/2*x3^3 - 5/4
x1 - 2*x3^4 + 1/2*x3^2 + 2/3
x2 + 2/3*x3^4 - 4/5*x3^3 - 2/3
x3^5 - 1/2*x3^3 + 1/3*x3


In [15]:
for g in G_drl:
    print(g)

x0^2 - 691/224*x0 - 17/48*x1 - 515/1792*x2 + 1/8*x3 + 2267/1008
x0*x1 + 127/84*x0 - x1 + 15/56*x2 - 1/12*x3 - 213/112
x1^2 - 13/14*x0 + 4/3*x1 - 65/112*x2 + 251/126
x0*x2 - 173/252*x0 + 1/10*x1 - 289/224*x2 - 2/45*x3 + 1199/1260
x1*x2 + 8/21*x0 - 4/9*x1 + 37/28*x2 + 2/15*x3 - 457/378
x2^2 - 214/315*x0 - 388/675*x1 - 157/84*x2 - 8/45*x3 + 17968/14175
x0*x3 - 39/56*x0 + 15/448*x2 - 3/2*x3 + 95/112
x1*x3 + 5/14*x0 - 45/112*x2 + 4/3*x3 - 5/28
x2*x3 + 47/105*x0 + 39/56*x2 - 8/9*x3 - 43/42
x3^2 + 24/7*x0 + 2*x1 + 15/7*x2 - 92/21


In [11]:
ideal(tmp_F).change_ring(new_ring).groebner_basis(algorithm='libsingular:stdfglm', prot=True)

std in (QQ),(x0,x1,x2,x3),(dp(4),C)
[65535:2]5(4)s6(3)s7ss8(4)s(5)s(6)s9(8)s(9)s(11)s10s(12)s(13)s(14)s(16)s(18)s(19)s(21)11--ss(22)s(24)s(26)s(27)s(29)s12(31)s(30)---s(27)s(29)s(30)s(32)s(34)-s(35)-s(36)ss13(39)-s(40)s(42)s(44)ss(46)s(47)s(48)s(50)s(52)s(53)-ss(55)s(57)-s(54)----s(52)s(53)s(54)s(56)s(58)s(60)s(62)s(64)----s(62)-s(61)s14(63)--s-s(64)-s(65)s(67)-s(68)s(70)s(72)s(71)s(70)s(71)--s(70)s(72)---s(70)s(72)s(74)s(76)s(78)s(80)s(82)s15(85)s(92)s(95)s(98)ss(101)s(103)s(104)s(107)s(110)-s(113)s(114)s(112)s(113)s(115)s(117)s(119)s(120)s(119)ss(121)s(116)--s(111)s(110)s(113)-s(111)-16---s(109)s(110)ss(112)sss(113)s(114)s(109)s(111)s(113)s(116)s(118)s(120)s(122)s(121)s(123)s(125)s(126)s(127)ss17(128)-----s(122)s(119)ss18(120)s19(114)ss(111)sss(108)s(109)s(110)s(108)-s20(106)s(104)--s(98)--s(96)ssss(94)s(92)s(91)ss21--s(90)s(89)ssss23s24-------------------25-------------------------------------26---------------------------------
(S:9)---------
product criterion:364 chain criterion:34

[x0^2 - 691/224*x0 - 17/48*x1 - 515/1792*x2 + 1/8*x3 + 2267/1008, x0*x1 + 127/84*x0 - x1 + 15/56*x2 - 1/12*x3 - 213/112, x1^2 - 13/14*x0 + 4/3*x1 - 65/112*x2 + 251/126, x0*x2 - 173/252*x0 + 1/10*x1 - 289/224*x2 - 2/45*x3 + 1199/1260, x1*x2 + 8/21*x0 - 4/9*x1 + 37/28*x2 + 2/15*x3 - 457/378, x2^2 - 214/315*x0 - 388/675*x1 - 157/84*x2 - 8/45*x3 + 17968/14175, x0*x3 - 39/56*x0 + 15/448*x2 - 3/2*x3 + 95/112, x1*x3 + 5/14*x0 - 45/112*x2 + 4/3*x3 - 5/28, x2*x3 + 47/105*x0 + 39/56*x2 - 8/9*x3 - 43/42, x3^2 + 24/7*x0 + 2*x1 + 15/7*x2 - 92/21]

++++++


In [23]:
'''
prot=True gives some info about the GB computation but the printing is not well synchronized.
'''

# for fid in sample_dict:
#     print(f'- ID: {[fid]} --------', flush=True)

#     F, G = sample_dict[fid]
#     ring = PolynomialRing(QQ, 4, 'x', order='lex')
#     start = time.time()
#     G_drl = ideal(F).groebner_basis(algorithm='libsingular:stdfglm', prot=True)

#     end = time.time()
#     print(f'[{fid:03}] {algorithm} -- {end-start:.3f} sec')
#     print('\n', flush=True)
#     # break
    

- ID: [19] --------
std in (QQ),(x0,x1,x2,x3),(dp(4),C)
[65535:2[019] libsingular:stdfglm -- 0.154 sec


]4(4)s7(3)ss(4)s8(6)ss9(7)s(9)s(11)s10(12)s(11)s(13)s(15)s(16)s(18)s11-s(16)s(18)--ss(19)s(20)s(22)s(24)s(26)s(27)s(29)s12(31)--s--s(30)s(32)s(34)s(36)s(37)----s(35)s(37)s(39)-----s(36)s(38)s(40)-s(41)13----s(39)s(41)s(43)s(48)-s(49)---s(48)-s(49)---s(53)s14(55)-s(56)---------s(49)s(51)s(53)s(55)s(57)ss(56)s(57)s(58)s(59)s(61)----15-s16(59)ss(63)s17(64)s(63)s(56)s(59)s(62)s18(64)s(62)s(60)19-----s(57)-s(55)-s20(56)s(58)s(59)sss---21--------------------22---------------------------23---------
(S:9)---------
product criterion:149 chain criterion:1240
.....++++++++++
vdim= 5
.....++----+----+----
- ID: [313] --------
[313] libsingular:stdfglm -- 0.031 sec


std in (QQ),(x0,x1,x2,x3),(dp(4),C)
[65535:2- ID: [578] --------
]7(5)s(4)ss9ss10(5)s(4)-s11sss(5)s12(6)s(7)s(9)s(11)s(12)ss13(13)s(14)s(15)s(16)s(18)s(19)ss(21)s(22)s(24)14-s(23)s(22)s(24)s(26)s(28)-s-s(29)s(31)s15(33)ssss(34)s(36)

In [31]:
print(sample_dict.keys())

dict_keys([19, 313, 578, 648, 705, 797, 368, 540, 899])


In [4]:
def print_prot(F, G, fid, algorithm='libsingular:stdfglm'):
    start = time.time()
    G_drl = ideal(F).groebner_basis(algorithm=algorithm, prot=True)
    end = time.time()
    print(f'[ID={fid:03}] {algorithm} / GB size = {len(G_drl)} / {end-start:.3f} sec')

In [46]:
fid = 19
print_prot(*sample_dict[fid], fid, algorithm='libsingular:stdfglm')

std in (QQ),(x0,x1,x2,x3),(dp(4),C)
[ID=019] libsingular:stdfglm / GB size = 4 / 0.153 sec
[65535:2]4(4)s7(3)ss(4)s8(6)ss9(7)s(9)s(11)s10(12)s(11)s(13)s(15)s(16)s(18)s11-s(16)s(18)--ss(19)s(20)s(22)s(24)s(26)s(27)s(29)s12(31)--s--s(30)s(32)s(34)s(36)s(37)----s(35)s(37)s(39)-----s(36)s(38)s(40)-s(41)13----s(39)s(41)s(43)s(48)-s(49)---s(48)-s(49)---s(53)s14(55)-s(56)---------s(49)s(51)s(53)s(55)s(57)ss(56)s(57)s(58)s(59)s(61)----15-s16(59)ss(63)s17(64)s(63)s(56)s(59)s(62)s18(64)s(62)s(60)19-----s(57)-s(55)-s20(56)s(58)s(59)sss---21--------------------22---------------------------23---------
(S:9)---------
product criterion:149 chain criterion:1240
.....++++++++++
vdim= 5
.....++----+----+----


In [47]:
fid = 313
print_prot(*sample_dict[fid], fid, algorithm='libsingular:stdfglm')

[ID=313] libsingular:stdfglm / GB size = 4 / 0.044 sec
std in (QQ),(x0,x1,x2,x3),(dp(4),C)
[65535:2]7(5)s(4)ss9ss10(5)s(4)-s11sss(5)s12(6)s(7)s(9)s(11)s(12)ss13(13)s(14)s(15)s(16)s(18)s(19)ss(21)s(22)s(24)14-s(23)s(22)s(24)s(26)s(28)-s-s(29)s(31)s15(33)ssss(34)s(36)ss(37)ss(38)ss(39)-s(40)s(42)s(43)s(45)s16(47)s(44)-s(41)s(39)sss(40)sss(42)s(44)s(46)s(48)s(49)ss(51)s17(53)-s-s(45)-s(44)s(43)s(45)s(47)s18(49)-s(45)s(47)-s(42)s(41)s(39)s(40)-s(41)ss(43)-s(42)s19(40)ss-s(41)s(42)s20(41)-s(42)---21------22----------------------23-----------
(S:9)---------
product criterion:95 chain criterion:816
.....++++++++++
vdim= 5
.....++----+----+----


In [48]:
fid = 578
print_prot(*sample_dict[fid], fid, algorithm='libsingular:stdfglm')

[ID=578] libsingular:stdfglm / GB size = 4 / 0.003 sec
std in (QQ),(x0,x1,x2,x3),(dp(4),C)
[65535:2]4(5)s(4)s5s(5)s6(6)s(7)s7(8)s(10)ss(11)s(13)s(14)s(15)s8(17)ss(19)s10(13)s(12)s11(11)--12--13-14------.16
(S:3)---
product criterion:13 chain criterion:67
..++++---
vdim= 2
..++-+-+-


In [49]:
fid = 648
print_prot(*sample_dict[fid], fid, algorithm='libsingular:stdfglm')

[ID=648] libsingular:stdfglm / GB size = 4 / 15.518 sec
std in (QQ),(x0,x1,x2,x3),(dp(4),C)
[65535:2]5(4)s6(3)s7ss8(4)s(5)s(6)s9(8)s(9)s(11)s10s(12)s(13)s(14)s(16)s(18)s(19)s(21)11--ss(22)s(24)s(26)s(27)s(29)s12(31)s(30)---s(27)s(29)s(30)s(32)s(34)-s(35)-s(36)ss13(39)-s(40)s(42)s(44)ss(46)s(47)s(48)s(50)s(52)s(53)-ss(55)s(57)-s(54)----s(52)s(53)s(54)s(56)s(58)s(60)s(62)s(64)----s(62)-s(61)s14(63)--s-s(64)-s(65)s(67)-s(68)s(70)s(72)s(71)s(70)s(71)--s(70)s(72)---s(70)s(72)s(74)s(76)s(78)s(80)s(82)s15(85)s(92)s(95)s(98)ss(101)s(103)s(104)s(107)s(110)-s(113)s(114)s(112)s(113)s(115)s(117)s(119)s(120)s(119)ss(121)s(116)--s(111)s(110)s(113)-s(111)-16---s(109)s(110)ss(112)sss(113)s(114)s(109)s(111)s(113)s(116)s(118)s(120)s(122)s(121)s(123)s(125)s(126)s(127)ss17(128)-----s(122)s(119)ss18(120)s19(114)ss(111)sss(108)s(109)s(110)s(108)-s20(106)s(104)--s(98)--s(96)ssss(94)s(92)s(91)ss21--s(90)s(89)ssss23s24-------------------25-------------------------------------26---------------------------------

In [51]:
F, G = sample_dict[19]
G_drl = ideal(F).groebner_basis(algorithm=algorithm)

In [56]:
for fid in sample_dict:
    F, G = sample_dict[fid]
    new_ring = PolynomialRing(QQ, 4, 'x', order='degrevlex')
    G_drl = ideal(G).change_ring(new_ring).groebner_basis(algorithm='libsingular:stdfglm')

    print(f'[ID={fid}], GB size = {len(G_drl)}')
    for g in G_drl:
        print(g)

    print('')


[ID=19], GB size = 10
x0^2 - 61/36*x0 - 197/45*x1 - 56/45*x2 - 1741/360*x3 + 2771/900
x0*x1 + 151/72*x0 + 7003/720*x1 + 871/180*x2 + 55147/2880*x3 - 10193/1800
x1^2 - 865/144*x0 - 21133/1440*x1 - 2401/360*x2 - 163777/5760*x3 + 1838/225
x0*x2 - 239/180*x0 - 689/180*x1 - 77/180*x2 - 3071/720*x3 + 383/900
x1*x2 + 127/72*x0 + 1621/180*x1 + 26/9*x2 + 2453/144*x3 - 313/90
x2^2 - 1619/288*x0 - 45527/2880*x1 - 2951/720*x2 - 406883/11520*x3 - 145/36
x0*x3 - 1/6*x0 - 29/12*x1 - 5/3*x2 - 269/48*x3 + 11/6
x1*x3 + 19/12*x0 + 319/120*x1 + 43/30*x2 + 2251/480*x3 - 161/75
x2*x3 + 4/3*x0 + 77/60*x1 - 1/15*x2 + 881/240*x3 + 154/75
x3^2 - x0 - 13/10*x1 - 2/5*x2 - 97/40*x3 + 8/25

[ID=313], GB size = 10
x0^2 + 22790989/309744*x0 + 2390054237/108410400*x1 - 54724387/1806840*x2 + 4256077579/21682080*x3 - 2354818357/20649600
x0*x1 + 1750598/19359*x0 + 32873767/1084104*x1 - 3835021/90342*x2 + 1434565889/5420520*x3 - 160491691/1032480
x1^2 + 1911625/19359*x0 + 7136383/135513*x1 - 4521550/45171*x2 + 101679281/2

In [57]:
fid = 578
print_prot(*sample_dict[fid], fid, algorithm='libsingular:std')

[ID=578] libsingular:std / GB size = 4 / 5.604 sec[65535:1]4(5)s(4)s5s(5)s6(6)s7
(5)s(6)s8(7)ss(8)s9(5)10.11.s(6)s12(7)s..13.s(8).s14(7).-...15.s(6)...16.s(7)s17.s(8)...18.ss(9)ss-19.s(7)sss.20.sssss.s21ssssss22sssssss23.sssssss24ssssssss25sssssss26ss(6)sssss27sssssss28ssss--29----
(S:3)---
product criterion:161 chain criterion:102


In [58]:
fid = 578
print_prot(*sample_dict[fid], fid, algorithm='libsingular:slimgb')

[ID=578] libsingular:slimgb / GB size = 4 / 0.196 sec
4M[2,2](6)5M[1,1](8)6M[2,2](12)7M[7,7](19)8M[13,v.v.v.v.9](24)4M[2,ev.1](26)7M[3,v.1](25)8M[7,ev.v.v.v.v.2](29)9M[10,bv.v.v.v.5](32)10M[4,v.v.2](33)10M[2,bv.1](36)11M[8,v.bv.v.v.v.v.v.1](38)11M[6,6](45)12M[24,vvvvv21](41)12M[6,v.v.v.v.2](43)12M[4,v.v.2](46)12M[2,2](49)13M[20,v.v.v.v.v.v.v.v.v.v.v.v.v.v.v.5](43)12M[1,bv.0](43)13M[15,bbv.v.v.v.v.v.v.v.v.v.v.4](46)13M[11,v.v.v.v.v.v.v.v.3](49)13M[8,8](54)9M[1,1](58)8M[1,1](62)7M[1,1](70)6M[1,1](76)5M[1,1](82)4M[1,1](88)8M[1,b0](77)11M[3,bbbbb0](63)12M[3,b0](56)13M[7,b0](47)14M[9,ev.ev.bbb0](13)13M[1,eeeeebb0](12)14M[1,0](11)17M[1,0](0)
NF:190 product criterion:829, ext_product criterion:0 
[65535:1]2(3)s1(2)sss
(S:3)---


In [59]:
print_prot(*sample_dict[fid], fid, algorithm='libsingular:stdfglm')

[ID=578] libsingular:stdfglm / GB size = 4 / 0.003 sec
std in (QQ),(x0,x1,x2,x3),(dp(4),C)
[65535:2]4(5)s(4)s5s(5)s6(6)s(7)s7(8)s(10)ss(11)s(13)s(14)s(15)s8(17)ss(19)s10(13)s(12)s11(11)--12--13-14------.16
(S:3)---
product criterion:13 chain criterion:67
..++++---
vdim= 2
..++-+-+-


In [ ]:
load_path = f'results/shape_gb_lex/gb_dataset_n={n}_field={field}'
bag = load_trained_bag(load_path, from_checkpoint=True)
model = bag['model'] 
tokenizer = bag['tokenizer']

NameError: name 'test_loader' is not defined

(array([], dtype=int64),)

In [ ]:
field = 'QQ'
for n in range(2, 6):
    print(f' field = {field}, n = {n}')
    load_dir = f'results/shape_gb_lex/gb_dataset_n={n}_field={field}'
    experiement(field, n, load_dir)
    print('\n')



# Katsura-n

In [55]:
def mylatex(f):
    return latex(f)

def load_katsura(field, n, return_ring=False):
    if field == 'QQ':
        field_ = QQ 
    if field[0] == 'F' and field[1:].isdigit():
        field_ = GF(int(field[1:]))
        
    ring = PolynomialRing(field_, n, 'x', order='lex')
    I = sage.rings.ideal.Katsura(ring, n)  
    F = I.basis
    G = I.groebner_basis()
    return (F, G, ring) if return_ring else (F, G)


def experiement(field, n, load_dir, num_beams=1):
    bag = load_trained_bag(load_dir, from_checkpoint=True)
    model = bag['model'] 
    tokenizer = bag['tokenizer']
    params = bag['params']
    
    F, G = load_katsura(field, n)
    F_prefix = [poly_to_prefix(f) for f in F]
    G_prefix = [poly_to_prefix(g) for g in G]
    x_text = ' [SEP] '.join(F_prefix)
    y_text = ' [SEP] '.join(G_prefix)

    x = tokenizer(x_text, return_tensors='pt')['input_ids'].cuda()
    # y = tokenizer(y_text, return_tensors='pt')['input_ids'].cuda()
    output_ids = model.generate(x, max_length=1000, num_beams=num_beams, do_sample=False)
    z_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    G_pred = [prefix_to_poly(zt, F.ring) for zt in z_text[0].split('[SEP]')]
    
    print(F)
    print(G)
    print(G_pred)

        

In [6]:
field = 'QQ'
for n in range(2, 6):
    print(f' field = {field}, n = {n}')
    load_dir = f'results/shape_gb_lex/gb_dataset_n={n}_field={field}'
    experiement(field, n, load_dir)
    print('\n')

 field = QQ, n = 2


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.
/home/kera/mambaforge/envs/sage/lib/python3.10/site-packages/optimum/bettertransformer/models/encoder_models.py:487: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, ~attention_mask)


[x0 + 2*x1 - 1, x0^2 - x0 + 2*x1^2]
[x0 + 2*x1 - 1, x1^2 - 1/3*x1]
[None, None]


 field = QQ, n = 3


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


[x0 + 2*x1 + 2*x2 - 1, x0^2 - x0 + 2*x1^2 + 2*x2^2, 2*x0*x1 + 2*x1*x2 - x1]
[x0 - 60*x2^3 + 158/7*x2^2 + 8/7*x2 - 1, x1 + 30*x2^3 - 79/7*x2^2 + 3/7*x2, x2^4 - 10/21*x2^3 + 1/84*x2^2 + 1/84*x2]
[None, None, None]


 field = QQ, n = 4


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


[x0 + 2*x1 + 2*x2 + 2*x3 - 1, x0^2 - x0 + 2*x1^2 + 2*x2^2 + 2*x3^2, 2*x0*x1 + 2*x1*x2 - x1 + 2*x2*x3, 2*x0*x2 + x1^2 + 2*x1*x3 - x2]
[x0 - 53230079232/1971025*x3^7 + 10415423232/1971025*x3^6 + 9146536848/1971025*x3^5 - 2158574456/1971025*x3^4 - 838935856/5913075*x3^3 + 275119624/5913075*x3^2 + 4884038/5913075*x3 - 1, x1 - 97197721632/1971025*x3^7 + 73975630752/1971025*x3^6 - 12121915032/1971025*x3^5 - 2760941496/1971025*x3^4 + 814792828/1971025*x3^3 - 1678512/1971025*x3^2 - 9158924/1971025*x3, x2 + 123812761248/1971025*x3^7 - 79183342368/1971025*x3^6 + 7548646608/1971025*x3^5 + 3840228724/1971025*x3^4 - 2024910556/5913075*x3^3 - 132524276/5913075*x3^2 + 30947828/5913075*x3, x3^8 - 8/11*x3^7 + 4/33*x3^6 + 131/5346*x3^5 - 70/8019*x3^4 + 1/3564*x3^3 + 5/42768*x3^2 - 1/128304*x3]
[None, None, None, None]


 field = QQ, n = 5


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


[x0 + 2*x1 + 2*x2 + 2*x3 + 2*x4 - 1, x0^2 - x0 + 2*x1^2 + 2*x2^2 + 2*x3^2 + 2*x4^2, 2*x0*x1 + 2*x1*x2 - x1 + 2*x2*x3 + 2*x3*x4, 2*x0*x2 + x1^2 + 2*x1*x3 + 2*x2*x4 - x2, 2*x0*x3 + 2*x1*x2 + 2*x1*x4 - x3]
[x0 - 308110355965692339884186470634147686371505891532807871811902272247524425728/2300416163855081226283764659441213559936734646783690132119987*x4^15 + 2745916744964843340073319972931317434499748758333498363131468420213845884928/16102913146985568583986352616088494919557142527485830924839909*x4^14 - 1331408292613478234836082577998991556829776751466175794713581464105517547520/16102913146985568583986352616088494919557142527485830924839909*x4^13 + 257402374905027595104460066253088484947405403489320457627961780169280561152/16102913146985568583986352616088494919557142527485830924839909*x4^12 + 19049599372039000321335526895674219707181251837096131891595208561971849216/16102913146985568583986352616088494919557142527485830924839909*x4^11 - 20585599662255306935074784913735873725428112366305860178

In [7]:
field = 'F7'
for n in range(2, 6):
    print(f' field = {field}, n = {n}')
    # load_dir = f'results/shape_gb_lex/gb_dataset_n={n}_field={field}'
    load_dir = f'results/shape_gb_lex/gb_dataset_n={n}_char=7'
    experiement(field, n, load_dir, num_beams=1)
    print('\n')

 field = F7, n = 2


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


[x0 + 2*x1 - 1, x0^2 - x0 + 2*x1^2]
[x0 + 2*x1 - 1, x1^2 + 2*x1]
[None, None]


 field = F7, n = 3


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


[x0 + 2*x1 + 2*x2 - 1, x0^2 - x0 + 2*x1^2 + 2*x2^2, 2*x0*x1 + 2*x1*x2 - x1]
[x0 + 2*x1 + 2*x2 - 1, x1^2 - 3*x1, x1*x2 + 2*x1, x2^2 + 2*x2]
[None, None, None]


 field = F7, n = 4


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


[x0 + 2*x1 + 2*x2 + 2*x3 - 1, x0^2 - x0 + 2*x1^2 + 2*x2^2 + 2*x3^2, 2*x0*x1 + 2*x1*x2 - x1 + 2*x2*x3, 2*x0*x2 + x1^2 + 2*x1*x3 - x2]
[x0 - 2*x3^6 + x3^5 - 2*x3^4 + 3*x3^3 + 3*x3^2 - 2*x3 - 1, x1 - 3*x3^6 + 2*x3^4 + x3^3 + 2*x3^2 - 3*x3, x2 - 3*x3^6 + 3*x3^5 - x3^4 + x3^3 - 2*x3, x3^7 + 3*x3^6 - x3^5 + 3*x3^4 + x3^3 - x3^2 + 3*x3]
[None, None, None, None]


 field = F7, n = 5


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


[x0 + 2*x1 + 2*x2 + 2*x3 + 2*x4 - 1, x0^2 - x0 + 2*x1^2 + 2*x2^2 + 2*x3^2 + 2*x4^2, 2*x0*x1 + 2*x1*x2 - x1 + 2*x2*x3 + 2*x3*x4, 2*x0*x2 + x1^2 + 2*x1*x3 + 2*x2*x4 - x2, 2*x0*x3 + 2*x1*x2 + 2*x1*x4 - x3]
[x0 + 2*x2 - 2*x4^13 + 2*x4^9 + x4^8 + 2*x4^7 - x4^6 - x4^4 - x4^3 + 3*x4^2 + x4 - 1, x1 - 2*x4^13 + 2*x4^12 - x4^11 + 3*x4^8 + x4^6 + 2*x4^5 - x4^4 + 2*x4^3 + 2*x4^2 - x4, x2^2 - 3*x2 - x4^13 + 2*x4^11 - 2*x4^10 - x4^9 - 3*x4^6 - x4^5 - 2*x4^4 + 2*x4^3 + 2*x4^2 - 3*x4, x2*x4 + 2*x2 + x4^12 + x4^11 - x4^10 + x4^9 + x4^8 + 2*x4^7 - x4^6 - x4^5 + 3*x4^4 + 2*x4^3 + 3*x4^2 + 3*x4, x3 + 3*x4^13 - 2*x4^12 + x4^11 - x4^9 - x4^7 + 3*x4^6 - 2*x4^5 - 2*x4^4 + 2*x4^3 - 2*x4, x4^14 - x4^13 - 2*x4^12 + x4^11 - 3*x4^10 - x4^9 + x4^8 - 2*x4^7 - x4^6 + x4^5 - 3*x4^4 + 3*x4^3 - 3*x4^2 - x4]
[None, None, None, None, None]




In [11]:
field = 'F31'
char = int(field[1:])
for n in range(2,4):
    print(f' field = {field}, n = {n}')
    # load_dir = f'results/shape_gb_lex/gb_dataset_n={n}_field={field}'
    load_dir = f'results/shape_gb_lex/gb_dataset_n={n}_char={char}'
    experiement(field, n, load_dir, num_beams=1)
    print('\n')

 field = F31, n = 2


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


[x0 + 2*x1 - 1, x0^2 - x0 + 2*x1^2]
[x0 + 2*x1 - 1, x1^2 + 10*x1]
[None, None]


 field = F31, n = 3


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


[x0 + 2*x1 + 2*x2 - 1, x0^2 - x0 + 2*x1^2 + 2*x2^2, 2*x0*x1 + 2*x1*x2 - x1]
[x0 + 2*x2^3 - 4*x2^2 + 10*x2 - 1, x1 - x2^3 + 2*x2^2 - 4*x2, x2^4 + x2^3 - 7*x2^2 - 7*x2]
[None, None, None]




In [15]:
for field in ['QQ']:
    for n in range(2, 6):
        F, G, ring = load_katsura(field, n, return_ring=True)
        
        for i in range(max(len(F), len(G))):
            f_text = f'f_{i+1} = {latex(F[i])}' if i < len(F) else ''
            g_text = f'g_{i+1} = {latex(G[i])}' if i < len(G) else ''
            
            print(f'{f_text:80} & {g_text} \\\\')

        print('')


f_1 = x_{0} + 2 x_{1} - 1                                                        & g_1 = x_{0} + 2 x_{1} - 1 \\
f_2 = x_{0}^{2} - x_{0} + 2 x_{1}^{2}                                            & g_2 = x_{1}^{2} - \frac{1}{3} x_{1} \\

f_1 = x_{0} + 2 x_{1} + 2 x_{2} - 1                                              & g_1 = x_{0} - 60 x_{2}^{3} + \frac{158}{7} x_{2}^{2} + \frac{8}{7} x_{2} - 1 \\
f_2 = x_{0}^{2} - x_{0} + 2 x_{1}^{2} + 2 x_{2}^{2}                              & g_2 = x_{1} + 30 x_{2}^{3} - \frac{79}{7} x_{2}^{2} + \frac{3}{7} x_{2} \\
f_3 = 2 x_{0} x_{1} + 2 x_{1} x_{2} - x_{1}                                      & g_3 = x_{2}^{4} - \frac{10}{21} x_{2}^{3} + \frac{1}{84} x_{2}^{2} + \frac{1}{84} x_{2} \\

f_1 = x_{0} + 2 x_{1} + 2 x_{2} + 2 x_{3} - 1                                    & g_1 = x_{0} - \frac{53230079232}{1971025} x_{3}^{7} + \frac{10415423232}{1971025} x_{3}^{6} + \frac{9146536848}{1971025} x_{3}^{5} - \frac{2158574456}{1971025} x_{3}^{4} - \frac{83893585

In [58]:
p = ring.random_element()
p

-x0^2 - 3*x0 + 2*x1^2 - 1

In [60]:
latex(p)

-x_{0}^{2} + 4 x_{0} + 2 x_{1}^{2} + 6

In [46]:
F, G, ring = load_katsura('F7', 2, return_ring=True)
F

[x0 + 2*x1 - 1, x0^2 - x0 + 2*x1^2]

In [54]:
latex(F)

\left[x_{0} + 2 x_{1} + 6, x_{0}^{2} - x_{0} + 2 x_{1}^{2}\right]

In [52]:
latex(F[0])

x_{0} + 2 x_{1} + 6

In [53]:
latex(F[1])

x_{0}^{2} - x_{0} + 2 x_{1}^{2}

In [44]:
for field in ['F7']:
    for n in range(2, 6):
        F, G, ring = load_katsura(field, n, return_ring=True)
        
        for i in range(max(len(F), len(G))):
            f_text = f'f_{i+1} = {latex(F[i])}' if i < len(F) else ''
            g_text = f'g_{i+1} = {latex(G[i])}' if i < len(G) else ''
            
            print(f'{f_text:80} & {g_text} \\\\')

        print('')


f_1 = x_{0} + 2 x_{1} + 6                                                        & g_1 = x_{0} + 2 x_{1} + 6 \\
f_2 = x_{0}^{2} - x_{0} + 2 x_{1}^{2}                                            & g_2 = x_{1}^{2} + 2 x_{1} \\

f_1 = x_{0} + 2 x_{1} + 2 x_{2} + 6                                              & g_1 = x_{0} + 2 x_{1} + 2 x_{2} + 6 \\
f_2 = x_{0}^{2} - x_{0} + 2 x_{1}^{2} + 2 x_{2}^{2}                              & g_2 = x_{1}^{2} + 4 x_{1} \\
f_3 = 2 x_{0} x_{1} + 2 x_{1} x_{2} - x_{1}                                      & g_3 = x_{1} x_{2} + 2 x_{1} \\
                                                                                 & g_4 = x_{2}^{2} + 2 x_{2} \\

f_1 = x_{0} + 2 x_{1} + 2 x_{2} + 2 x_{3} + 6                                    & g_1 = x_{0} + 5 x_{3}^{6} + x_{3}^{5} + 5 x_{3}^{4} + 3 x_{3}^{3} + 3 x_{3}^{2} + 5 x_{3} + 6 \\
f_2 = x_{0}^{2} - x_{0} + 2 x_{1}^{2} + 2 x_{2}^{2} + 2 x_{3}^{2}                & g_2 = x_{1} + 4 x_{3}^{6} + 2 x_{3}^{4} + x_{3}^{3}

In [17]:
for field in ['F31']:
    for n in range(2, 6):
        F, G = load_katsura(field, n)
        
        for i in range(max(len(F), len(G))):
            f_text = f'f_{i+1} = {latex(F[i])}' if i < len(F) else ''
            g_text = f'g_{i+1} = {latex(G[i])}' if i < len(G) else ''
            
            print(f'{f_text:80} & {g_text} \\\\')

        print('')


f_1 = x_{0} + 2 x_{1} + 30                                                       & g_1 = x_{0} + 2 x_{1} + 30 \\
f_2 = x_{0}^{2} - x_{0} + 2 x_{1}^{2}                                            & g_2 = x_{1}^{2} + 10 x_{1} \\

f_1 = x_{0} + 2 x_{1} + 2 x_{2} + 30                                             & g_1 = x_{0} + 2 x_{2}^{3} + 27 x_{2}^{2} + 10 x_{2} + 30 \\
f_2 = x_{0}^{2} - x_{0} + 2 x_{1}^{2} + 2 x_{2}^{2}                              & g_2 = x_{1} - x_{2}^{3} + 2 x_{2}^{2} + 27 x_{2} \\
f_3 = 2 x_{0} x_{1} + 2 x_{1} x_{2} - x_{1}                                      & g_3 = x_{2}^{4} + x_{2}^{3} + 24 x_{2}^{2} + 24 x_{2} \\

f_1 = x_{0} + 2 x_{1} + 2 x_{2} + 2 x_{3} + 30                                   & g_1 = x_{0} + 22 x_{3}^{7} + 19 x_{3}^{6} + 26 x_{3}^{5} + 22 x_{3}^{4} + 17 x_{3}^{3} + 27 x_{3}^{2} + 13 x_{3} + 30 \\
f_2 = x_{0}^{2} - x_{0} + 2 x_{1}^{2} + 2 x_{2}^{2} + 2 x_{3}^{2}                & g_2 = x_{1} + 15 x_{3}^{7} + 12 x_{3}^{6} + 24 x_{3}^{5} + 29 x_{3}

In [6]:
a = PolynomialRing(QQ, 2, 'x', order='lex')


In [9]:
b = ideal(a).basis

In [12]:
b.ring()

Multivariate Polynomial Ring in x0, x1 over Rational Field

In [4]:
train_dataset, test_dataset = load_data('data/gb_dataset_n=2_field=QQ/data', encoding='lex.prefix', return_dataloader=False)

loading ... data/gb_dataset_n=2_field=QQ/data.train
loading ... data/gb_dataset_n=2_field=QQ/data.test


In [23]:
for batch in test_dataset:
    x_text, y_text = batch['input'], batch['target']
    break

In [18]:
tokenizer(x, return_tensors='pt')

{'input_ids': tensor([[210, 205,   0, 208, 105, 109, 213, 205, 206, 208,  99, 108, 206, 207,
           0, 107,   1, 205, 206, 208, 103, 108, 206, 207,   0, 106,   1, 205,
         206, 103, 206,   0, 207,   1, 107, 206, 208, 108, 109, 207,   1, 107,
         211]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [29]:
x = tokenizer(x_text, return_tensors='pt')['input_ids'].cuda()
y = tokenizer(y_text, return_tensors='pt')['input_ids'].cuda()
output_ids = model.generate(x, max_length=y.shape[-1], num_beams=1, do_sample=False)
z_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
z_text

['+ x0 / 1 5 [SEP] x1 [SEP] x2']

In [48]:
field = "QQ"

if field == 'QQ':
    field_ = QQ 
if field[0] == 'F' and field[1:].isdigit():
    field_ = GF(int(field[1:]))

n = 4
ring = PolynomialRing(field_, n, 'x', order='lex')
I = sage.rings.ideal.Katsura(ring,n)  
F = I.basis
G = I.groebner_basis()
print(F)
G

[x0 + 2*x1 + 2*x2 + 2*x3 - 1, x0^2 - x0 + 2*x1^2 + 2*x2^2 + 2*x3^2, 2*x0*x1 + 2*x1*x2 - x1 + 2*x2*x3, 2*x0*x2 + x1^2 + 2*x1*x3 - x2]


[x0 - 53230079232/1971025*x3^7 + 10415423232/1971025*x3^6 + 9146536848/1971025*x3^5 - 2158574456/1971025*x3^4 - 838935856/5913075*x3^3 + 275119624/5913075*x3^2 + 4884038/5913075*x3 - 1, x1 - 97197721632/1971025*x3^7 + 73975630752/1971025*x3^6 - 12121915032/1971025*x3^5 - 2760941496/1971025*x3^4 + 814792828/1971025*x3^3 - 1678512/1971025*x3^2 - 9158924/1971025*x3, x2 + 123812761248/1971025*x3^7 - 79183342368/1971025*x3^6 + 7548646608/1971025*x3^5 + 3840228724/1971025*x3^4 - 2024910556/5913075*x3^3 - 132524276/5913075*x3^2 + 30947828/5913075*x3, x3^8 - 8/11*x3^7 + 4/33*x3^6 + 131/5346*x3^5 - 70/8019*x3^4 + 1/3564*x3^3 + 5/42768*x3^2 - 1/128304*x3]

In [49]:
save_dir = f'results/shape_gb_lex/gb_dataset_n={n}_field={field}'
bag = load_trained_bag(save_dir, from_checkpoint=True)
model = bag['model'] 
tokenizer = bag['tokenizer']
params = bag['params']

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [50]:
F_prefix = [poly_to_prefix(f) for f in F]
G_prefix = [poly_to_prefix(g) for g in G]
intext = ' [SEP] '.join(F_prefix)
outtext = ' [SEP] '.join(G_prefix)

In [51]:
inseq = tokenizer.encode(intext)
inseq = torch.tensor(inseq).reshape(1, -1).cuda()
inseq

tensor([[210, 205,   0, 205, 206, 106,   1, 205, 206, 106,   2, 205, 206, 106,
           3, 103, 213, 205, 207,   0, 106, 205, 206, 103,   0, 205, 206, 106,
         207,   1, 106, 205, 206, 106, 207,   2, 106, 206, 106, 207,   3, 106,
         213, 205, 206, 106, 206,   0,   1, 205, 206, 106, 206,   1,   2, 205,
         206, 103,   1, 206, 106, 206,   2,   3, 213, 205, 206, 106, 206,   0,
           2, 205, 207,   1, 106, 205, 206, 106, 206,   1,   3, 206, 103,   2,
         211]], device='cuda:0')

In [52]:
output_ids = model.generate(inseq, max_length=100, num_beams=1, do_sample=False)
z_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
[prefix_to_poly(zt,ring) for zt in z_text[0].split('[SEP]')]

[x0 - 1, x1, x2 - 1, x3]

In [111]:
predseq = output.logits.argmax(dim=int(-1))
pred_prefix = tokenizer.decode(predseq[0])
print(pred_prefix)
pred = [prefix_to_poly(prefix, ring) for prefix in pred_prefix.split('[SEP]')]
pred

+ + x0 [SEP] 1 5 [SEP] x1 x1 / 4 3 ^ / x3 [SEP] ^ [SEP] x2 / 1 [SEP] ^ / x3 [SEP] ^ [SEP] x2 / ^ / ^ x3 / [SEP] x2 1 5 [SEP] x3 [SEP] ^ [PAD]


[None, 5, None, None, None, None, None, None, 5, x3, None]

In [61]:
pred

[x2, x2, x2, x2, x2, -1, -1, -1, 2, -1]

In [15]:
# save_dir = 'results/shape_gb/gb_n=2_r=5_d=3_m=2_Gd=5_Gm=None_F=4/'
save_dir = 'results/shape_gb2/gb_dataset_n=3_char=7'
# save_dir = 'results/shape_gb/gb_dataset_n=5_char=7'
bag = load_trained_bag(save_dir, from_checkpoint=True)
model = bag['model'] 
tokenizer = bag['tokenizer']
params = bag['params']

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [16]:
from src.loader.data import load_data
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

batch_size = 4
encoding = 'prefix'
data_path = 'data3/gb_dataset_n=3_char=7/data'
testset = load_data(data_path, encoding=encoding, extensions=['test'], do_shuffle=[False], return_dataloader=False)
testloader = load_data(data_path, encoding=encoding, batch_sizes=[batch_size], extensions=['test'], do_shuffle=[False], return_dataloader=True, tokenizer=tokenizer)
# batch = next(iter(testloader))

In [17]:
results = eval_prediction(model, testloader, use_tqdm=True, num_beams=1)
results

100%|██████████| 250/250 [01:28<00:00,  2.83it/s]


{'acc': 0.505,
 'hits': [tensor([ True,  True, False, False]),
  tensor([False,  True,  True, False]),
  tensor([ True, False, False,  True]),
  tensor([ True,  True,  True, False]),
  tensor([False, False, False, False]),
  tensor([False, False,  True,  True]),
  tensor([ True, False,  True, False]),
  tensor([False,  True, False,  True]),
  tensor([False,  True, False,  True]),
  tensor([True, True, True, True]),
  tensor([ True, False, False,  True]),
  tensor([ True, False, False, False]),
  tensor([ True,  True,  True, False]),
  tensor([False,  True, False, False]),
  tensor([False,  True,  True, False]),
  tensor([ True,  True, False, False]),
  tensor([False,  True, False, False]),
  tensor([ True,  True, False,  True]),
  tensor([False,  True, False, False]),
  tensor([False,  True, False, False]),
  tensor([ True, False, False, False]),
  tensor([False, False, False, False]),
  tensor([ True, False, False, False]),
  tensor([False,  True,  True, False]),
  tensor([False,  Tru

In [7]:
testset

TypeError: 'NoneType' object is not subscriptable

In [47]:

for hits, batch in zip(results['hits'], testloader):
    x, y = batch['input_ids'], batch['decoder_input_ids']
    x, y = x.cuda(), y.cuda()
    
    output_ids = model.generate(x, max_length=y.shape[-1], num_beams=1, do_sample=False)
    
    x_text = tokenizer.batch_decode(x, skip_special_tokens=True)
    y_text = tokenizer.batch_decode(y, skip_special_tokens=True)
    z_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    
    for hit, yt, zt in zip(hits, y_text, z_text):
        print(f"{'   '}", yt)
        print(f"{'[!]' if hit else '   '}", zt)
        print('')


    + x0 ** x1 3 [SEP] * 3 ** x1 3
[!] + x0 ** x1 3 [SEP] * 3 ** x1 3

    + 3 + x0 + * -1 ** x1 3 * -3 ** x1 5 [SEP] + ** x1 5 * -3 x1
    + 3 + x0 + * -1 ** x1 3 * -3 ** x1 5 [SEP] + ** x1 5 * 2 x1

    + 1 + x0 + * -1 ** x1 4 * -3 ** x1 5 [SEP] + 3 + * -1 ** x1 4 + * -2 x1 * 2 ** x1 5
    + 3 + x0 + * -1 ** x1 4 + * -3 ** x1 5 * 2 x1 [SEP] + 2 + * -1 ** x1 4 + * -3 ** x1 5 * 2 ** x1 3

    + -3 + x0 + x1 + ** x1 5 * 2 ** x1 4 [SEP] + ** x1 5 + * -3 ** x1 2 + * -2 ** x1 4 * 2 x1
    + -3 + x0 + x1 + ** x1 5 * 2 ** x1 4 [SEP] + * -3 ** x1 2 + * -3 ** x1 4 * 2 ** x1 5

    + -2 + x0 + * -1 ** x1 3 + * -2 x1 + * 3 ** x1 4 * 3 ** x1 5 [SEP] + -1 + * -2 ** x1 4 + * 2 ** x1 2 * 3 ** x1 5
    + -2 + x0 + x1 + * -1 ** x1 3 + * 3 ** x1 4 * 3 ** x1 5 [SEP] + -1 + * -2 ** x1 4 + * 2 ** x1 2 * 3 ** x1 5

    + 3 x0 [SEP] * -1 ** x1 4
[!] + 3 x0 [SEP] * -1 ** x1 4

    + -2 + x0 + ** x1 3 + * -2 ** x1 4 * 2 ** x1 5 [SEP] + * -1 ** x1 5 + * -3 x1 + * -3 ** x1 2 * 2 ** x1 3
    + -2 + x0 + ** x1 3 

In [7]:
from tqdm import tqdm
from time import time 

In [8]:
%%time

from tqdm import tqdm
tot, acc = 0, 0
hits_list = []
z_text_list = []
with torch.no_grad():
    for batch in tqdm(testloader):
        x, y = batch['input_ids'].cuda(), batch['decoder_input_ids'].cuda()
        output_ids = model.generate(x, max_length=y.shape[-1], num_beams=1, do_sample=False)
        z_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        z_text_list.append(z_text)
        
        l = min(y.shape[-1], output_ids.shape[-1])
        hits = torch.all(y[:, :l] == output_ids[:, :l], dim=1)
        hits_list.append(hits)
        acc += sum(hits).item()
        tot += len(hits)

    acc /= tot
    print(f'acc = {acc:.4f}')

  0%|          | 0/125 [00:00<?, ?it/s]/home/kera/anaconda3/lib/python3.9/site-packages/optimum/bettertransformer/models/encoder_models.py:487: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, ~attention_mask)
100%|██████████| 125/125 [00:39<00:00,  3.20it/s]

acc = 0.3430
CPU times: user 38.8 s, sys: 532 ms, total: 39.4 s
Wall time: 39.1 s


In [30]:
for batch, hits in zip(testloader, hits_list):
    x, y = batch['input_ids'].cuda(), batch['decoder_input_ids'].cuda()
    x_text = tokenizer.batch_decode(x, skip_special_tokens=True)
    y_text = tokenizer.batch_decode(y, skip_special_tokens=True)
    
    for i, hit in enumerate(hits):
        if hit:
            print(x_text[i])
            print('--> ', y_text[i])
            
    break

+ * -3 ** x0 3 + * 3 ** x1 3 * -3 * ** x0 2 ** x1 3 [SEP] + x0 + ** x1 3 + * 2 ** x0 4 + * -2 * x0 ** x1 3 * 2 * ** x0 3 ** x1 3
-->  + x0 ** x1 3 [SEP] * 3 ** x1 3
+ * -3 ** x1 7 + * 2 ** x1 3 + * x1 ** x0 2 + * -1 * ** x0 2 ** x1 6 + * -2 * x1 ** x0 3 * 2 * ** x0 2 ** x1 4 [SEP] + 3 + x0 + * -1 ** x1 3 + * -3 ** x1 5 + * x0 ** x1 9 + * -3 * x0 ** x1 5 + * -3 * ** x0 3 ** x1 6 + * -2 * ** x0 3 ** x1 8 + * 2 * ** x0 3 ** x1 3 * 3 * ** x0 4 ** x1 3 [SEP] + ** x1 5 + * -1 ** x1 4 + * -3 ** x1 6 + * x0 x1 + * ** x0 4 ** x1 3 + * -1 * ** x0 2 ** x1 2 + * -1 * ** x0 3 ** x1 6 + * -3 * x0 ** x1 2 + * -3 * ** x0 3 ** x1 8 + * -2 * x0 ** x1 9 + * 3 * x0 ** x1 7 * 3 * ** x0 3 ** x1 3
-->  + 3 + x0 + * -1 ** x1 3 * -3 ** x1 5 [SEP] + ** x1 5 * -3 x1
+ * -1 ** x1 4 + * ** x0 2 ** x1 2 * 3 * x0 ** x1 2 [SEP] + 3 + x0 + * -1 ** x1 7 + * ** x0 2 ** x1 5 * 3 * x0 ** x1 5
-->  + 3 x0 [SEP] * -1 ** x1 4
+ * -1 ** x1 4 + * -1 ** x1 6 * x0 ** x1 2 [SEP] + x0 + * -1 ** x1 4 + * 3 ** x1 5 + * 3 ** x1 7 * -

In [34]:
for batch, hits, z_text in zip(testloader, hits_list, z_text_list):
    x, y = batch['input_ids'].cuda(), batch['decoder_input_ids'].cuda()
    x_text = tokenizer.batch_decode(x, skip_special_tokens=True)
    y_text = tokenizer.batch_decode(y, skip_special_tokens=True)
    
    for i, hit in enumerate(hits):
        if not hit:
            print(x_text[i])
            print('--> ', z_text[i])
            print('GT: ', y_text[i])
            print('')
            
    break

+ * -1 ** x1 3 + * -1 ** x1 5 + * -3 ** x1 8 + * -2 x1 + * -2 ** x1 7 + * 3 ** x1 2 + * 3 ** x1 4 + * x0 ** x1 6 + * -1 * x0 x1 + * -1 * x0 ** x1 2 + * -1 * ** x0 2 ** x1 2 * 3 * x0 ** x1 7 [SEP] + 2 + x1 + * -1 ** x1 2 + * -3 ** x1 7 + * -3 ** x1 9 + * -2 ** x1 6 + * -2 ** x1 8 + * 2 ** x1 5 + * 3 ** x1 3 + * 3 ** x1 4 + * x0 ** x1 6 + * -1 * x0 ** x1 2 + * -1 * x0 ** x1 3 + * -1 * ** x0 2 ** x1 3 + * -3 * x0 ** x1 5 + * -2 * ** x0 2 ** x1 2 + * 2 * x0 x1 * 3 * x0 ** x1 8 [SEP] + -2 + ** x1 8 + * -3 ** x0 2 + * -3 ** x0 3 + * -3 ** x1 4 + * -3 ** x1 5 + * -2 ** x1 6 + * -2 ** x1 7 + * 2 x0 + * 2 x1 + * 2 ** x1 2 + * 2 ** x1 3 + * ** x0 2 ** x1 6 + * ** x0 3 ** x1 2 + * ** x0 3 ** x1 3 + * ** x0 3 ** x1 4 + * ** x0 4 ** x1 3 + * -1 * x0 ** x1 6 + * -1 * x0 ** x1 7 + * -1 * ** x0 2 ** x1 3 + * -1 * ** x0 2 ** x1 5 + * -1 * ** x0 3 ** x1 7 + * -3 * ** x0 2 ** x1 4 + * -3 * ** x0 3 ** x1 8 + * -2 * x0 ** x1 9 + * -2 * ** x0 2 ** x1 7 + * -2 * ** x0 2 ** x1 8 + * 2 * x0 ** x1 8 + * 2 * ** 

: 

In [11]:
for c in [7]:
    for n in range(2, 6):
        save_dir = f'results/shape_gb/gb_dataset_n={n}_char={c}'
        bag = load_trained_bag(save_dir, from_checkpoint=False)
        model = bag['model'] 
        tokenizer = bag['tokenizer']
        params = bag['params']
        
        data_path = f'data/gb_dataset_n={n}_char={c}/data'
        # testset = load_data(data_path, extensions=['test'], do_shuffle=[False], return_dataloader=False)
        testloader = load_data(data_path, batch_sizes=[64], extensions=['test'], do_shuffle=[False], return_dataloader=True, tokenizer=tokenizer)
        
        tot, acc = 0, 0
        with torch.no_grad():
            start_time = time()
            for batch in tqdm(testloader):
                x, y = batch['input_ids'].cuda(), batch['decoder_input_ids'].cuda()
                output_ids = model.generate(x, max_length=y.shape[-1], num_beams=1, do_sample=False)
                # z_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
                
                l = min(y.shape[-1], output_ids.shape[-1])
                hits = torch.all(y[:, :l] == output_ids[:, :l], dim=1)
                acc += hits.sum().item()
                tot += len(hits)

            acc /= tot
            runtime = time() - start_time 
            print(f'(n={n}, char={c}): acc = {acc:.4f}, runtime = {runtime:.2f} [sec]')
        

100%|██████████| 16/16 [00:08<00:00,  1.85it/s]


(n=2, char=7): acc = 0.4140, runtime = 8.63 [sec]


100%|██████████| 16/16 [00:14<00:00,  1.08it/s]


(n=3, char=7): acc = 0.6200, runtime = 14.88 [sec]


100%|██████████| 16/16 [00:19<00:00,  1.22s/it]


(n=4, char=7): acc = 0.7480, runtime = 19.54 [sec]


100%|██████████| 16/16 [00:22<00:00,  1.40s/it]

(n=5, char=7): acc = 0.7960, runtime = 22.46 [sec]


In [22]:
n, c = 2, 31
save_dir = f'results/shape_gb/gb_dataset_n={n}_char={c}'
bag = load_trained_bag(save_dir, from_checkpoint=False)
model = bag['model'] 
tokenizer = bag['tokenizer']
params = bag['params']

data_path = f'data/gb_dataset_n={n}_char={c}/data'
# testset = load_data(data_path, extensions=['test'], do_shuffle=[False], return_dataloader=False)
testloader = load_data(data_path, batch_sizes=[64], extensions=['test'], do_shuffle=[False], return_dataloader=True, tokenizer=tokenizer)

tot, acc = 0, 0
with torch.no_grad():
    start_time = time()
    for batch in tqdm(testloader):
        x, y = batch['input_ids'].cuda(), batch['decoder_input_ids'].cuda()
        output_ids = model.generate(x, max_length=y.shape[-1], num_beams=1, do_sample=False)
        # z_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
        
        l = min(y.shape[-1], output_ids.shape[-1])
        hits = torch.all(y[:, :l] == output_ids[:, :l], dim=1)
        acc += hits.sum().item()
        tot += len(hits)

    acc /= tot
    runtime = time() - start_time 
    print(f'(n={n}, char={c}): acc = {acc:.4f}, runtime = {runtime:.2f} [sec]')


'+ x0 ** x1 3 [SEP] * 3 ** x1 3'

In [21]:
z_text

['+ x0 ** x1 3 [SEP] * 3 ** x1 3']

In [6]:
input_text, output_text = batch['input'], batch['target']

KeyError: 'input'

In [17]:
eval_prediction(model, testset, tokenizer, max_samples=10, use_tqdm=False, num_beams=5)

+ x0 ** x1 3 [SEP] * 3 ** x1 3
+
------------

+ 3 + x0 + * -1 ** x1 3 * -3 ** x1 5 [SEP] + ** x1 5 * -3 x1
+
------------

+ 2 + x0 + * -3 x1 + * 2 ** x1 4 * 3 ** x1 5 [SEP] + 2 + * -1 ** x1 4 + * -3 ** x1
+
------------

+ -3 + x0 + x1 + * -3 ** x1 4 * -3 ** x1 5 [SEP] + * -3 ** x1 2 + * -2 ** x1 4 * 2 x1
+
------------

+ -2 + x0 + ** x1 4 + * -1 ** x1 3 + * -2 x1 * -2 ** x1 5 [SEP] + -1 + * -2 ** x1 4 + * 2 ** x1 2 * 3 ** x1 5
+
------------

+ 3 x0 [SEP] * -1 ** x1 4
+
------------

+ -2 + x0 + ** x1 3 + * -2 ** x1 4 * 2 ** x1 5 [SEP] + * -3 ** x1 2 + * -3 ** x1 5 + * -2 ** x1 4 * 2 **
+
------------

+ -2 + x0 + * -1 ** x1 4 + * -3 ** x1 2 + * -3 ** x1 5 * 3 ** x1 3 [SEP] + -2 + x1 + * -1 ** x1 4 + * -3 ** x1
+
------------

+ x0 + * -1 ** x1 2 + * -1 ** x1 3 + * -1 ** x1 5 + * -3 ** x1 4 * -2 x1 [SEP] + 3 + * -1 ** x1 3 + * -1 ** x1 5 + * 2
+
------------

+ -2 + x0 + * -3 x1 + * -3 ** x1 2 * 2 ** x1 4 [SEP] + x1 + * -1 ** x1 5 + * -2 ** x1 4 + * 3 ** x1 2 * 3 ** x1 3
+
--------

{'acc': 0.0,
 'hits': [False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False],
 'num_beams': 5}

In [6]:
x = tokenizer(input_text, padding=True, return_tensors='pt').input_ids
y = tokenizer(output_text, padding=True, return_tensors='pt').input_ids
x, y = x.cuda(), y.cuda()

In [7]:
max_length=x.shape[-1]
output_ids = model.generate(x, max_length=max_length, num_beams=1, do_sample=False)

In [33]:
pred_text = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
pred_text

['+ x0 + ** x1 3 + * -1 x1 + * -1 ** x1 2 * -2 ** x1 4 [SEP] + -1 + * -1 ** x1 4 + * 2 ** x1 2 * 2 ** x1 5',
 '+ 1 + x0 + ** x1 5 + * -1 x1 + * 2 ** x1 2 * 2 ** x1 4 [SEP] + ** x1 2 + * -1 x1 + * -2 ** x1 3 + * 2 ** x1 4 * 2 ** x1 5',
 '+ 1 + x0 + ** x1 5 + * -1 x1 + * 2 ** x1 2 * 2 ** x1 4 [SEP] + ** x1 2 + * -1 x1 + * -2 x1 + * -2 ** x1 3 + * -1 ** x1 4 * -2 ** x1 5',
 '+ -1 + x0 + ** x1 5 + * -1 x1 + * 2 ** x1 2 * 2 ** x1 4 [SEP] + ** x1 2 + * -1 x1 + * -1 ** x1 3 + * 2 ** x1 4 * 2 ** x1 5']

In [36]:
len(trainloader)

1000000

In [9]:
batch = next(iter(testloader))
batch

{'data': ['+ x0 + * -1 ** x1 3 + * -1 ** x1 4 + * -1 ** x1 5 + * -2 ** x1 2 * 2 x1 [SEP] + -1 + * -1 ** x1 3 + * -1 ** x1 5 + * -2 ** x1 6 + * -2 ** x1 7 + * -2 ** x1 8 + * -1 * x0 ** x1 4 + * -2 * x0 ** x1 3 + * -2 * ** x0 2 ** x1 2 + * 2 * x0 ** x1 5 + * 2 * x0 ** x1 6 * 2 * x0 ** x1 7'],
 'target': ['+ x0 + * -1 ** x1 3 + * -1 ** x1 4 + * -1 ** x1 5 + * -2 ** x1 2 * 2 x1 [SEP] + -1 + ** x1 4 + * -1 ** x1 3 * -2 ** x1 5']}

In [4]:
## ==============================
##       Overlap test 
## ==============================

from tqdm import tqdm
from src.loader.data import load_data
data_path = 'data/gb_n=2_r=5_d=3_m=2_Gd=5_Gm=None_F=4'
trainloader, testloader = load_data(data_path, batch_sizes=[1,1000])

count = 0
count_c = 0
total = 100
for i, batch in enumerate(tqdm(trainloader)):
    if i > total: break
    x, y = batch['data'], batch['target']
    x, y = x[0], y[0]
    for batch_ in testloader:
        x_, y_ = batch_['data'], batch_['target']
        
        if x in x_ or y in y_: 
            count += 1
            # count_c += int(x == x_ and y == y_)
            break 

print(f'{count} / {total} overlap') # 0 / 100 overlap
# print(f'{count_c} / {total} complete overlap')

  0%|          | 101/1000000 [01:08<188:49:07,  1.47it/s]

0 / 100 overlap


In [5]:
x

'+ x0 + ** x1 2 + ** x1 5 + * -1 x1 + * 2 ** x1 3 * 2 ** x1 4 [SEP] + -1 + x0 + ** x0 3 + ** x1 2 + ** x1 5 + * -1 x1 + * -2 ** x1 4 + * 2 ** x1 3 + * x0 ** x1 5 + * x0 ** x1 6 + * ** x0 2 ** x1 5 + * -1 * x1 ** x0 2 + * -1 * ** x0 2 ** x1 2 + * -2 * x0 ** x1 4 + * -2 * x0 ** x1 7 + * 2 * x0 ** x1 3 + * 2 * ** x0 2 ** x1 3 * 2 * ** x0 2 ** x1 4'

In [34]:
for pred, target in zip(pred_text, output_text):
    print(pred == target)

False
False
False
False


In [ ]:
pred_text[0]

In [ ]:
output_text[0]

In [ ]:
i = 3
pred_text[i][:len(output_text[i])] == output_text[i]

In [3]:
from src.data.tokenizers import set_vocab, set_tokenizer

In [ ]:
tokenizer.vocab

In [ ]:
data_path = 'data/gb_n=2_r=5_d=3_m=2_Gd=5_Gm=None_F=4'
trainloader, testloader = load_data(data_path)


In [ ]:
batch = next(iter(testloader))
x, y = batch['data'], batch['target']

In [ ]:
vocab = set_vocab('none', 3, max_int=100)
tok = set_tokenizer(vocab)

In [ ]:
vocab

In [ ]:
tok(['-1 1'])

In [ ]:
a = tok(x, y, return_tensors='pt', padding='longest')

In [ ]:
a = tok(x, y, return_tensors='pt', padding='longest', verbose=True)
b = tok(list(zip(x, y)), return_tensors='pt', padding='longest', verbose=True)

In [ ]:
len(a.input_ids)

In [ ]:
len(b.input_ids[0])

In [ ]:
a.token_type_ids

In [ ]:
len(b.input_ids[0])

In [ ]:
b = tok(x[0], y[0], return_tensors='pt', padding='longest')

In [ ]:
b.token_type_ids

In [ ]:
from src.data.symbolic_utils import * 
from src.utils.utils import *


In [ ]:
import torch
import yaml 
import os 
import argparse
import re 

save_dir = 'results/shape_gb/_gb_n=2_r=5_d=3_m=2_Gd=5_Gm=None_F=4/'

def load_args(save_dir):
    config_file = os.path.join(save_dir, 'params.yaml')
    with open(config_file, 'r') as f:
        config = yaml.safe_load(f)
    args = argparse.Namespace(**config)
    return args 

def get_checkpoint_id(save_dir):
    cpt_file = [f for f in os.listdir(save_dir) if 'checkpoint' in f][0]
    cpid = int(re.search(r'checkpoint-(\d+)', cpt_file).group(1))
    return cpid 

from transformers import PreTrainedTokenizerFast
def load_tokenizer(save_dir):
    cpid = get_checkpoint_id(save_dir)
    tokenizer = PreTrainedTokenizerFast.from_pretrained(os.path.join(save_dir, f'tokenizer.json'))
    return tokenizer

def load_pretrained_model(save_dir, params, tokenizer):
    # from src.loader._models import Transformer
    model = Transformer(tokenizer, 
                        encoder_layers = params.encoder_layers, 
                        decoder_layers = params.decoder_layers, 
                        attention_heads = params.attention_heads, 
                        embedding_dimension=params.embedding_dimension, 
                        max_sequence_length=params.max_seq_length, 
                        feedforward_dimension=params.feedforward_dimension,
                        dropout=params.dropout
                        ) 

    # print(model.generate)

    model_path = os.path.join(save_dir, 'pytorch_model.bin')
    model_config = torch.load(model_path, map_location='cpu')
    model.load_state_dict(model_config)
    
    # model.from_pretrained(model_path)
    return model 

def load_trained_bag(save_dir):
    cpid = get_checkpoint_id(save_dir)
    checkpoint_path = os.path.join(save_dir, f'checkpoint-{cpid}')
    
    params = load_args(save_dir)
    tokenizer = load_tokenizer(save_dir)
    model = load_pretrained_model(checkpoint_path, params, tokenizer)
    
    bag = {'model': model, 'params': params, 'tokenizer': model.tokenizer}
    
    return bag


In [ ]:
bag = load_trained_bag(save_dir)
model = bag['model'].cuda() 
tokenizer = bag['tokenizer']
params = bag['params']

In [ ]:
train_dataset, test_dataset = load_data(params.data_path, return_dataloader=False)

In [ ]:
model.return_loss=False

In [ ]:
x = tokenizer(input_text, return_tensors='pt').input_ids.cuda()
y = tokenizer(out_text, return_tensors='pt').input_ids.cuda()

In [ ]:
mean_acc = 0
model.return_loss=False
for i in range(100):
    input_text = test_dataset[0]['data']
    out_text = test_dataset[0]['target']

    x = tokenizer(input_text, return_tensors='pt').input_ids.cuda()
    y = tokenizer(out_text, return_tensors='pt').input_ids.cuda()

    attention_mask = ~tokenizer(input_text, return_tensors='pt')['attention_mask'].bool().cuda()
    decoder_attention_mask = ~tokenizer(out_text, return_tensors='pt')['attention_mask'].bool().cuda()

    z = model(x, y, attention_mask=attention_mask, decoder_attention_mask=decoder_attention_mask)

    acc = (z.argmax(dim=-1) == y).float().mean().item()
    mean_acc += acc 
    # print(acc)
    # decoder_attention_mask=None
    
print(mean_acc)

In [ ]:
from pytorch_beam_search.seq2seq import greedy_search

In [ ]:
from transformers.generation_utils import greedy_search

In [ ]:
tokenizer.batch_decode(predictions)

In [ ]:
predictions

In [ ]:
log_probabilities.shape

In [ ]:
tokenizer = bag['tokenizer']

In [ ]:
len(tokenizer.vocab)

In [ ]:
tokenizer.special_tokens_map

In [12]:
vocab = set_vocab(2, field_char=0, max_int=20)
tokernizer = set_tokenizer(vocab)

In [8]:
len(tok.vocab)

52

In [9]:
tok.vocab_size

52

In [11]:
len(tok.get_vocab())

52

In [23]:
from src.data.tokenizers import set_tokenizer, set_vocab

In [25]:
vocab = set_vocab(3)
tok = set_tokenizer(vocab, max_seq_length=1024)

In [26]:
tok.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]'}

In [ ]:
print(len(tok.vocab))
tok.vocab.keys()

In [ ]:
for k in tokenizer.vocab:
    if k not in tok.vocab:
        print(k)

In [ ]:
tokenizer.vocab_size

In [ ]:
tok.vocab_size

In [ ]:
tokenizer.special_tokens_map

In [ ]:
tok.special_tokens_map

In [ ]:
params = load_args(save_dir)
tokenizer = load_tokenizer(save_dir)
cpid = get_checkpoint_id(save_dir)

In [ ]:
# from src.loader.model import load_model
from src.loader._models import Transformer

model = Transformer(tokenizer, 
                    encoder_layers = params.encoder_layers, 
                    decoder_layers = params.decoder_layers, 
                    attention_heads = params.attention_heads, 
                    embedding_dimension=params.embedding_dimension, 
                    max_sequence_length=params.max_seq_length, 
                    feedforward_dimension=params.feedforward_dimension,
                    dropout=params.dropout
                    ) 
    

In [ ]:
import torch
# model_path = os.path.join(save_dir, 'pytorch_model.bin')
model_path = os.path.join(save_dir, f'checkpoint-{cpid}/pytorch_model.bin')
model_config = torch.load(model_path, map_location='cpu')
model.load_state_dict(model_config)

In [ ]:
# set up tokenizer
cpid = get_checkpoint_id(save_dir)
tokenizer = PreTrainedTokenizerFast.from_pretrained(os.path.join(save_dir, f'checkpoint-{cpid}'))
tokenizer.add_special_tokens({'pad_token': '[PAD]', 'cls_token': '[CLS]', 'bos_token': '<s>', 'eos_token': '</s>'})
from tokenizers.processors import TemplateProcessing
tokenizer.post_processor = TemplateProcessing(
    single="<s> $A </s>",
    special_tokens=[("<s>", tokenizer.bos_token_id), ("</s>", tokenizer.eos_token_id)],
)

# load model 
config = BartConfig.from_pretrained(os.path.join(save_dir, f'checkpoint-{cpid}/config.json'))
model = BartForConditionalGeneration.from_pretrained(os.path.join(save_dir, f'checkpoint-{cpid}/pytorch_model.bin'), config=config)
model.eval().cuda()


In [ ]:
def reload_model(path, args):
    with open(path) as file:
        params = yaml.safe_load(file)
    corpus = read_file(data_path + '.corpus')

    model = Transformer(env, env, 
                        encoder_layers = params['encoder_layers'], 
                        decoder_layers = params['decoder_layers'], 
                        attention_heads = params['attention_heads'], 
                        embedding_dimension=params['embedding_dimension'], 
                        max_sequence_length=params['max_seq_length'], 
                        feedforward_dimension=params['feedforward_dimension'],
                        dropout=params['dropout'])

    model.print_architecture()
    sd = torch.load(os.path.join(params['save_path'], 'best_model.pth'), map_location='cpu')
    model.load_state_dict(sd)
    
    return model 

In [ ]:
import torch 
import torch.nn as nn 
import math 
from torch import Tensor 
from transformers import GenerationMixin, PreTrainedModel
from attrdict import AttrDict

class Transformer(nn.Module):
        
    def __init__(self, 
                tokenizer,
                max_sequence_length = 1024,
                embedding_dimension = 512,
                feedforward_dimension = 2048,
                encoder_layers = 6,
                decoder_layers = 6,
                attention_heads = 8,
                activation = "gelu",
                dropout = 0.1,
                ):

        super().__init__()
        
        max_sequence_length = 1024*2
        
        self.return_loss = True
        self.batch_first = True
        # # float16 / distributed (AMP)
        # if params.amp >= 0:
        #     self.init_amp()
        #     if params.multi_gpu:
        #         for k in self.modules.keys():
        #             self.modules[k] = apex.parallel.DistributedDataParallel(self.modules[k], delay_allreduce=True)
        self.tokenizer = tokenizer
        self.vocab_size = 51 # len(tokenizer.vocab)  # tokenizer.vocab_size does not include special tokens
        self.generation_config = AttrDict({'bos_token_id': tokenizer.cls_token_id, 
                                           'eos_token_id': tokenizer.sep_token_id})
        self.pad_token_id = tokenizer.pad_token_id
            
        self.source_embeddings = nn.Embedding(self.vocab_size, embedding_dimension)
        self.target_embeddings = self.source_embeddings
        self.positional_embeddings =  PositionalEncoding(d_model=embedding_dimension, dropout=dropout, max_len=max_sequence_length)

        self.transformer = nn.Transformer(d_model = embedding_dimension, 
                                          dim_feedforward = feedforward_dimension,
                                          nhead = attention_heads, 
                                          num_encoder_layers = encoder_layers, 
                                          num_decoder_layers = decoder_layers,
                                          activation = activation,
                                          dropout = dropout,
                                          batch_first=True)
        
        self.output_layer = nn.Linear(embedding_dimension, self.vocab_size)
        
        self.architecture = dict(model = "Seq2Seq Transformer",
                                 vocab = self.tokenizer.vocab,
                                 max_sequence_length = max_sequence_length,
                                 embedding_dimension = embedding_dimension,
                                 feedforward_dimension = feedforward_dimension,
                                 encoder_layers = encoder_layers,
                                 decoder_layers = decoder_layers,
                                 attention_heads = attention_heads,
                                 activation = activation,
                                 dropout = dropout)

    def criterion(self, z, y):
        loss_fn = nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
        return loss_fn(z.reshape(-1, z.shape[-1]), y.flatten())

    def print_architecture(self):
        """
        Displays the information about the model in standard output. 
        """
        for k in self.architecture.keys():
            print(f"{k.replace('_', ' ').capitalize()}: {self.architecture[k]}")
        print(f"Trainable parameters: {sum([p.numel() for p in self.parameters()]):,}")
        print(flush=True)
        
    def tokenize(self, x):
        return self.tokenizer(x, padding=True, truncation=False, return_tensors='pt').input_ids

    def encode(self, X, mask):
        '''
        X: batch_size x seq_length
        '''
        assert(X.shape[0] <= self.architecture["max_sequence_length"])
        src_pad_mask = (X == self.pad_token_id)
        
        X = self.source_embeddings(X) #* math.sqrt(self.architecture['embedding_dimension'])
        X = self.positional_embeddings(X)

        return self.transformer.encoder(X, src_key_padding_mask=src_pad_mask)
    
    def decode(self, encoder_outputs, Y): 
        assert Y.shape[1] <= self.architecture["max_sequence_length"]

        mask = self.transformer.generate_square_subsequent_mask(Y.shape[1]).to(Y.device)
        tgt_pad_mask = (Y == self.pad_token_id)
        
        Y = self.target_embeddings(Y) #* math.sqrt(self.architecture['embedding_dimension'])
        Y = self.positional_embeddings(Y)
                
        return self.transformer.decoder(Y, encoder_outputs, tgt_mask=mask, tgt_key_padding_mask=tgt_pad_mask)

    def forward(self, input_ids=None, decoder_input_ids=None, attention_mask=None, decoder_attention_mask=None, labels=None, return_loss=None):
        X, Y = input_ids, decoder_input_ids
        """
        Forward method of the model.
        
        Parameters
        ----------
        X: LongTensor of shape (batch_size, input_length)
            Tensor of integers containing the inputs for the model.
            
        Y: LongTensor of shape (batch_size, output_length)
            Tensor of integers containing the output produced so far.
            
        Returns
        -------
        output: FloatTensor of shape (batch_size, output_length, len(out_vocabulary))
            Tensor of floats containing the inputs for the final Softmax layer (usually integrated in the loss function).
        """
        # assert X.shape[1] <= self.architecture["max_sequence_length"]
        # assert Y.shape[1] <= self.architecture["max_sequence_length"]
        
        src_pad_mask = (X == self.pad_token_id)
        tgt_pad_mask = (Y == self.pad_token_id)

        X_embd = self.source_embeddings(X) / math.sqrt(self.architecture['embedding_dimension'])
        X_embd = self.positional_embeddings(X_embd)
        
        Y_embd = self.target_embeddings(Y) / math.sqrt(self.architecture['embedding_dimension'])
        Y_embd = self.positional_embeddings(Y_embd)

        mask = self.transformer.generate_square_subsequent_mask(Y_embd.shape[1]).to(Y_embd.device)
        transformer_output = self.transformer(src=X_embd, tgt=Y_embd, 
                                              tgt_mask=mask, 
                                              src_key_padding_mask=attention_mask, 
                                              tgt_key_padding_mask=decoder_attention_mask)
        outputs = self.output_layer(transformer_output) # (batch_size, seq_len, vocab_size)
        
        return_loss = self.return_loss if return_loss is None else return_loss
        loss = self.criterion(outputs, Y) if return_loss else None
        outputs = outputs if self.batch_first else outputs.permute(1, 0, 2)
        
        return (loss, outputs) if return_loss else outputs
        # return outputs
    
    def generate(self, X, prediction_length=30, method='greedy_search'):
        '''
        X: batch_size x length
        '''
        batch_size = X.shape[0]
        device = next(model.parameters()).device
        bos = self.generation_config.bos_token_id
        eos = self.generation_config.eos_token_id

        print(bos)

        with torch.no_grad():
            Y = bos * torch.ones(batch_size, 1).long().to(device)
            probabilities = torch.zeros(batch_size).to(device)
        
            iterator = range(prediction_length)
            for i in iterator:
                next_probabilities = model.forward(X, Y)[:, -1].log_softmax(-1)
                max_next_probabilities, next_chars = next_probabilities.max(-1)
                next_chars = next_chars.unsqueeze(-1)
                Y = torch.cat((Y, next_chars), axis = 1)
                probabilities += max_next_probabilities
        return Y, probabilities 
        


class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 1024):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        pe = pe.permute(1, 0, 2)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [batch_size, seq_len, embedding_dim]
        """
        x = x + self.pe[:, :x.shape[1], :]
        return self.dropout(x)
    

In [ ]:
bag = load_trained_bag(save_dir)
model = bag['model'].cuda() 
tokenizer = bag['tokenizer']
params = bag['params']
model.return_loss = False

In [ ]:
X = torch.ones(1, 2000).long().cuda()
model.max_sequence_length = 2000
out = model(X, X)

In [ ]:
x

In [ ]:
predictions, log_probabilities = model.generate(x[:, 1:])
tokenizer.batch_decode(predictions)

In [ ]:
model(x, y).argmax(dim=-1)

In [ ]:
y

In [ ]:
def greedy_search(
    model, 
    X, 
    predictions = 20,
    progress_bar = False
):
    """
    Implements Greedy Search to compute the output with the sequences given in X. The method can compute 
    several outputs in parallel with the first dimension of X.
    Parameters
    ----------    
    X: LongTensor of shape (examples, length)
        The sequences to start the decoding process.
    predictions: int
        The number of tokens to append to X.
    progress_bar: bool
        Shows a tqdm progress bar, useful for tracking progress with large tensors.
    Returns
    -------
    Y: LongTensor of shape (examples, length + predictions)
        The output sequences.
    probabilities: FloatTensor of length examples
        The estimated log-probabilities for the output sequences. They are computed by iteratively adding the 
        probability of the next token at every step.
    """
    # print('input:  ', X[0])
    with torch.no_grad():
        Y = torch.ones(X.shape[0], 1).long().to(next(model.parameters()).device)
        probabilities = torch.zeros(X.shape[0]).to(next(model.parameters()).device)
    
        iterator = range(predictions)
        if progress_bar:
            iterator = tqdm(iterator)
        for i in iterator:
            # print('output: ', Y)
            next_probabilities = model.forward(X, Y)[:, -1].log_softmax(-1)
            max_next_probabilities, next_chars = next_probabilities.max(-1)
            next_chars = next_chars.unsqueeze(-1)
            Y = torch.cat((Y, next_chars), axis = 1)
            probabilities += max_next_probabilities
    return Y, probabilities

In [ ]:
from transformers import BartForConditionalGeneration, BartConfig
import torch 


In [ ]:
config = BartConfig(
    encoder_layers=params.encoder_layers,
    encoder_attention_heads=params.attention_heads,
    decoder_layers=params.decoder_layers,
    decoder_attention_heads=params.attention_heads,
    vocab_size=51, 
    d_model=params.embedding_dimension, 
    encoder_ffn_dim=params.feedforward_dimension,
    decoder_ffn_dim=params.feedforward_dimension,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    max_position_embeddnigs=params.max_seq_length
)
model = BartForConditionalGeneration(config).cuda()


In [ ]:
X = torch.cat([x for _ in range(10)], dim=1).cuda()
X.shape

In [ ]:
model(X)